Goal is to investigate using PCA before doing the kmeans distance. Maybe compare with random projection (which should be worth although aren't there guarantees on distance preservation that may be relevant? idk, totally out of my league here)

In [ ]:
using Revise

using PotentialLearning, InteratomicPotentials
using Unitful
using Random
using AtomsBase
using DelimitedFiles
using Statistics: mean, var
using StatsBase
using Clustering, Distances
using Trapz
using LinearAlgebra: Symmetric, eigen, mul!, svd, cond, dot, norm

using MultivariateStats, StatsAPI

#using CairoMakie CairoMakie.activate!()
using GLMakie; GLMakie.activate!(inline=false)

In [ ]:
ace = ACE(species           = [:C,:H,:O,:N],
          body_order        = 3,
          polynomial_degree = 10,
          wL                = 2.0,
          csp               = 1.0,
          r0                = 1.43,
          rcutoff           = 4.4 )
lb = LBasisPotential(ace)
length(ace)

In [ ]:
qm9_file = "../files/QM9/qm9_fullset_alldata.xyz"
raw_data = load_data(qm9_file, ExtXYZ(u"eV", u"Å"))
raw_data = DataSet([config for config in raw_data if !(:F in atomic_symbol(get_system(config)))])

In [ ]:
max_num_train = 120_001
master_perm_idxs = readdlm("./primary_permutation.txt", Int64)
possible_training_idxs = master_perm_idxs[1:max_num_train]
possible_test_idxs = master_perm_idxs[max_num_train+1:end]

num_train = 40_000
train_idxs = possible_training_idxs[1:num_train]

In [ ]:
lb.β .= readdlm("qm9_4elem_3body_poly10_fit40K.txt", Float64)

In [ ]:
etest_ref = get_all_energies(raw_data[possible_test_idxs])

etest_local_descrs = compute_local_descriptors(raw_data[possible_test_idxs],lb.basis)
ds_test = DataSet(raw_data[possible_test_idxs] .+ etest_local_descrs)
etest_pred = get_all_energies(ds_test,lb)

num_atoms_test = length.(get_system.(raw_data[possible_test_idxs]))

In [ ]:
@show e_mae, e_rmse, e_rsq = calc_metrics(etest_pred,etest_ref)

In [ ]:
function compute_mean_features(ds)
    mean_feature_perconfig = Vector{Float64}[]
    for (i,config) in enumerate(ds)
        if i % 100 == 0
            println(i)
        end
        mean_feature = mean(InteratomicPotentials.compute_local_descriptors(get_system(config), lb.basis))
        push!(mean_feature_perconfig,mean_feature)
    end

    reduce(hcat,mean_feature_perconfig)
end

# can either average over the distances between centers, or find the minimum
# could also vary the distance here...
function heuristic_uncertainty1(mean_feature_vec, km)
    #dist = mean(mapslices(x->Distances.euclidean(mean_feature_vec,x), km.centers, dims=1))
    dist = minimum(mapslices(x->Distances.euclidean(mean_feature_vec,x), km.centers, dims=1))
end

function heuristic_uncertainty2(mean_feature_vec, km)
    dist = mean(mapslices(x->Distances.euclidean(mean_feature_vec,x), km.centers, dims=1))
    #dist = minimum(mapslices(x->Distances.euclidean(mean_feature_vec,x), km.centers, dims=1))
end

function cosine_similarity(x,y)
    return dot(x,y)/((norm(x)*norm(y)))
end

function heuristic_uncertainty3(mean_feature_vec, km)
    dist = 1 - maximum(mapslices(x->cosine_similarity(mean_feature_vec,x), km.centers, dims=1))
    return dist
end

# useless because cosine similarity can be negative
function heuristic_uncertainty4(mean_feature_vec, km)
    dist = 1 - mean(mapslices(x->cosine_similarity(mean_feature_vec,x), km.centers, dims=1))
    return dist
end

In [ ]:
mean_train_features = compute_mean_features(raw_data[train_idxs])
mean_test_features  = compute_mean_features(raw_data[possible_test_idxs])

dt = StatsBase.fit(ZScoreTransform, mean_train_features, dims=2)
std_mean_train_features = StatsBase.transform(dt,mean_train_features)
std_mean_test_features = StatsBase.transform(dt,mean_test_features)

In [ ]:
x = std_mean_train_features
x[:,1] * x[:,1]'

In [ ]:
# manual, non-SVD implementation, doing it myself here just so I know what's happening
# data is assumed to be n x d
function my_incomplete_pca(data)
    d,n = size(data)
    XXt = zeros((d,d))
    for col in eachcol(data)
        XXt .+= col * col'
    end

    mean_XXt = XXt ./ n
end

function my_incomplete_pca2(data)
    d,n = size(data)
    XXt = zeros((d,d))
    # Pre-allocate the temporary vector for outer product
    temp = Vector{Float64}(undef, d)

    for col in eachcol(data)
        # In-place outer product accumulation
        @views for i in 1:d
            temp[i] = col[i]
            for j in i:d  # Use symmetry to reduce computations
                XXt[i,j] += temp[i] * col[j]
            end
        end
        # Fill in the lower triangle using symmetry
        for i in 1:d
            for j in 1:i-1
                XXt[i,j] = XXt[j,i]
            end
        end
    end

    XXt ./= n
    return XXt
end


In [ ]:
@time ver1 = my_incomplete_pca(x[:,1:1000])
@time ver2 = my_incomplete_pca2(x[:,1:1000])
ver1 ≈ ver2

These two cells were just clarify that the below doesn't work, though actually... I should be able to do a matrix product. yea nevermind this works too 

In [ ]:
my_d, my_n = size(x)
my_XXT = Matrix{Float64}(undef,my_d,my_d)
mul!(my_XXT, x[:,1:1000], x[:,1:1000]')
my_XXT ./= my_n

In [ ]:
my_x = x[:,1:1000]
my_xt = x[:,1:1000]'
my_x * my_xt ./ my_n # ahh ok so this does match

In [ ]:
function my_pca(data)
    d,n = size(data)
    XXt = zeros((d,d))
    # Pre-allocate the temporary vector for outer product
    temp = Vector{Float64}(undef, d)

    for col in eachcol(data)
        # In-place outer product accumulation
        @views for i in 1:d
            temp[i] = col[i]
            for j in i:d  # Use symmetry to reduce computations
                XXt[i,j] += temp[i] * col[j]
            end
        end
        # Fill in the lower triangle using symmetry
        for i in 1:d
            for j in 1:i-1
                XXt[i,j] = XXt[j,i]
            end
        end
    end

    mean_XXt = XXt ./ n

    @show cond(mean_XXt)


    eigen(Symmetric(mean_XXt))
end

function my_pca2(data)
    d,n = size(data)
    XXt = zeros((d,d))

    mul!(XXt, data, data')

    mean_XXt = XXt ./ n

    @show cond(mean_XXt)

    eigen(Symmetric(mean_XXt))
end

In [ ]:
#lambdas, evecs = my_pca(x)

In [ ]:
alt_lambda, alt_evecs = my_pca2(x)

In [ ]:
show(stdout, "text/plain",lambdas[1:100])

In [ ]:
#plot_lambda1 = [ val > 0 ? val : eps() for val in lambdas[end:-1:1]]
plot_lambda2 = [  val > 0 ? val : eps() for val in alt_lambda[end:-1:1]]

In [ ]:
#f = plot(log10.(plot_lambda1))
plot(log10.(plot_lambda2))
#display(f)

In [ ]:
function svd_pca(data)
    #F = svd(data; full=true) # takes too long
    F = svd(data)
    evecs = F.V
    svs = F.S
    #svs, evecs
    F
end

In [ ]:
#lambdas2, evecs2 = svd_pca(x)
F = svd_pca(x)


In [ ]:
M1 = StatsAPI.fit(MultivariateStats.PCA, x; mean=0)

In [ ]:
M2 = StatsAPI.fit(MultivariateStats.PCA,x; method=:svd)

In [ ]:
M1.proj

In [ ]:
pca_std_train_features = StatsAPI.predict(M1, std_mean_train_features)
pca_std_test_features = StatsAPI.predict(M1, std_mean_test_features)

In [ ]:
train_features = pca_std_train_features
#num_neighbors = 20
km = kmeans(train_features, 20, distance=Distances.Euclidean(), rng=Xoshiro(1))
km_10 = kmeans(train_features, 10, distance=Distances.Euclidean(), rng=Xoshiro(1))
km_50  = kmeans(train_features, 50, distance=Distances.Euclidean(), rng=Xoshiro(1))

km_200 = kmeans(train_features, 200, distance=Distances.Euclidean(), rng=Xoshiro(1))
km_2000 = kmeans(train_features, 2000, distance=Distances.Euclidean(), rng=Xoshiro(1))

In [ ]:
test_feature_distances1 = mapslices(vec -> heuristic_uncertainty1(vec, km), pca_std_test_features, dims=1)
test_feature_distances2 = mapslices(vec -> heuristic_uncertainty2(vec, km), pca_std_test_features, dims=1)

test_feature_distances3 = mapslices(vec -> heuristic_uncertainty1(vec, km_10), pca_std_test_features, dims=1)
test_feature_distances4 = mapslices(vec -> heuristic_uncertainty1(vec, km_50), pca_std_test_features, dims=1)


test_feature_distances5 = mapslices(vec -> heuristic_uncertainty3(vec, km_50), pca_std_test_features, dims=1)
test_feature_distances6 = mapslices(vec -> heuristic_uncertainty3(vec, km_50), pca_std_test_features, dims=1)


test_feature_distances7 = mapslices(vec -> heuristic_uncertainty3(vec, km_200), pca_std_test_features, dims=1)
test_feature_distances8 = mapslices(vec -> heuristic_uncertainty1(vec, km_200), pca_std_test_features, dims=1)

test_feature_distances9 = mapslices(vec -> heuristic_uncertainty3(vec, km_2000), pca_std_test_features, dims=1)
test_feature_distances10 = mapslices(vec -> heuristic_uncertainty1(vec, km_2000), pca_std_test_features, dims=1)

test_feature_distances11 = mapslices(vec -> heuristic_uncertainty4(vec, km_2000), pca_std_test_features, dims=1)

In [ ]:
for center in eachcol(km_2000.centers)
    @show cosine_similarity(pca_std_test_features[:,1],center)
end

In [ ]:
test_feature_distances = test_feature_distances11

In [ ]:
hist(test_feature_distances[1,:], bins=1000)

In [ ]:
fraction_calib = 0.1
peratom = true
alpha = 0.05
num_calib = floor(Int64, fraction_calib*length(possible_test_idxs))
num_test = length(possible_test_idxs) - num_calib

#idxs_wrt_test = Random.randperm(length(possible_test_idxs))
idxs_wrt_test = collect(1:length(possible_test_idxs))

calib_idxs_wrt_test = idxs_wrt_test[1:num_calib]
test_idxs_wrt_test = idxs_wrt_test[num_calib+1:end]

if !peratom
    calib_scores = abs.(etest_pred[calib_idxs_wrt_test] .- etest_ref[calib_idxs_wrt_test]) ./ test_feature_distances[calib_idxs_wrt_test]
    test_abs_residuals = abs.(etest_pred[test_idxs_wrt_test] .- etest_ref[test_idxs_wrt_test])

else
    calib_scores = ( abs.(etest_pred[calib_idxs_wrt_test] .- etest_ref[calib_idxs_wrt_test])
                    ./ num_atoms_test[calib_idxs_wrt_test] ./ test_feature_distances[calib_idxs_wrt_test] )
    test_abs_residuals = abs.(etest_pred[test_idxs_wrt_test] .- etest_ref[test_idxs_wrt_test]) ./ num_atoms_test[test_idxs_wrt_test]

end

q_hat = quantile(calib_scores, ceil((num_calib+1)*(1-alpha))/num_calib)

In [ ]:
qhat_scores = q_hat*test_feature_distances[test_idxs_wrt_test]
coverage = sum(test_abs_residuals .> qhat_scores) / num_test

In [ ]:
test_residuals = (etest_pred[test_idxs_wrt_test] .- etest_ref[test_idxs_wrt_test]) ./num_atoms_test[test_idxs_wrt_test]
#hist(abs.(etest_pred[test_idxs_wrt_test] .- etest_ref[test_idxs_wrt_test]) ./num_atoms_test[test_idxs_wrt_test],bins=100)
hist(test_residuals,bins=1000)

In [ ]:
function uncertainty_vs_residuals(uncertainty, residuals;
    title  ="Uncertainty vs. residuals",
    xlabel = "Distance",
    ylabel = "Residuals",
    figsize = (600,600))

    fig = Figure(size=(600,600))
    ax  = Axis(fig[1,1],
    title="Residuals vs distances",
    xlabel="Distances",
    ylabel="Residuals",
    #limits=(0,100,-0.01,0.03))
    limits=(0,1,-0.01,0.03))

    hlines!(ax, 0.0, color=:red, linestyle=:dash)

    scatter!(ax, uncertainty, residuals, markersize=5)

    #ax.aspect=DataAspect()
    fig
end



#scatter(test_feature_distances[test_idxs_wrt_test], test_abs_residuals)
uncertainty_vs_residuals(test_feature_distances[test_idxs_wrt_test], test_abs_residuals) # idk why this isn't plotting correctly

In [ ]:
xvals = test_feature_distances[test_idxs_wrt_test]
yvals = test_abs_residuals
A = hcat(ones(length(xvals)), xvals)
coeffs = A \ yvals
@show coeffs
y_pred = coeffs[2] .*xvals .+ coeffs[1]

ymean = mean(yvals)
ss_total = sum((yvals .- ymean).^2)
ss_residual = sum((yvals .- y_pred).^2)
r_squared = 1 - (ss_residual / ss_total)

# not very meaningful for bounded between 0 and 1 ? I guess it's still should be fine but the R2 is much worse.

In [ ]:
abs_res = test_abs_residuals
@show length(abs_res)
for bin_start in 0.000:0.001:0.015
    low = bin_start
    high = bin_start + 0.001
    idxs = [i for i in eachindex(abs_res) if abs_res[i] >= low && abs_res[i] < high]
    local_coverage = 1-sum(abs_res[idxs] .> qhat_scores[idxs])/length(idxs)
    println("$(low)-$(high) : $(length(idxs)) configs with coverage $(local_coverage)")
end
low = 0.012
high = 0.02
idxs = [i for i in eachindex(abs_res) if abs_res[i] >= low && abs_res[i] < high]
local_coverage = 1-sum(abs_res[idxs] .> qhat_scores[idxs])/length(idxs)
println("$(low)-$(high) : $(length(idxs)) configs with coverage $(local_coverage)")

local_coverage = 1 - sum(abs_res .> qhat_scores)/length(abs_res)
println("overall coverage is $(local_coverage)")

In [ ]:
sharpness = 2*mean(qhat_scores)

In [ ]:
var(qhat_scores)

In [ ]:
hist(qhat_scores,bins=1000)

In [ ]:
alpha_complements = collect(range(0.01,0.99,step=0.01))
alpha_refs = 1 .- alpha_complements

alpha_refs = collect(range(0.01,0.99,step=0.01))

predicted_alphas = Float64[]
#for ac in alpha_complements
#    alpha = 1-ac
for alpha in alpha_refs
    qh = quantile(calib_scores, clamp(ceil((num_calib+1)*(1-alpha))/num_calib, 0.0, 1.0))

    qh_scores = qh*test_feature_distances[test_idxs_wrt_test]
    predicted_alpha = sum(test_abs_residuals .> qh_scores) / num_test
    push!(predicted_alphas, predicted_alpha)
end

In [ ]:
make_calibration_plot(alpha_refs,predicted_alphas)

In [ ]:
function compute_miscalibration_area(expected_ps, observed_ps)
    area = 0.0
    #for i in 2:length(expected_ps)-1
    #    trap = abs(trapz(expected_ps[i-1:i+1], observed_ps[i-1:i+1]) -
    #             trapz(expected_ps[i-1:i+1], expected_ps[i-1:i+1]))
    for i in 2:length(expected_ps)
        trap = abs(trapz(expected_ps[i-1:i], observed_ps[i-1:i]) -
                 trapz(expected_ps[i-1:i], expected_ps[i-1:i]))
        area += trap
    end
    area
end

# converted from Medford jupyter notebook via Claude
function make_calibration_plot(expected_ps, observed_ps; width=600)
    # Convert to percentages
    expected_ps = expected_ps .* 100
    observed_ps = observed_ps .* 100

    fig = Figure(resolution=(width, width))
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
        xlabel="Expected conf. level",
        ylabel="Observed conf. level",
        limits=(0, 100, 0, 100)
    )

    # Main line
    lines!(ax, 1.0 .- expected_ps, observed_ps)

    # Diagonal reference line
    lines!(ax, 1.0 .-expected_ps, 1.0 .-expected_ps, linestyle=:dash, alpha=0.4)

    # Filled area between curves
    band!(ax, expected_ps, expected_ps, observed_ps, color=(:blue, 0.2))

    # Configure ticks - approximately 4 ticks on each axis
    ax.xticks = 0:10:100
    ax.yticks = 0:10:100

    # Add percentage signs to ticks
    ax.xtickformat = xs -> ["$(Int(x))%" for x in xs]
    ax.ytickformat = xs -> ["$(Int(x))%" for x in xs]

    ## Add text for miscalibration area
    #text!(ax, "miscalc. area = $(round(area, digits=3))",
    #    position=(8, 2),
    #    align=(:left, :bottom)
    #)

    return fig
end

# Claude
function parity_plot(etest_ref, etest_pred, qhat_scored;
                     title="Parity Plot",
                     xlabel="Reference Values",
                     ylabel="Predicted Values",
                     figsize=(600, 600))
    # Create figure and axis
    fig = Figure(size=figsize)
    ax = Axis(fig[1, 1],
    title=title,
    xlabel=xlabel,
    ylabel=ylabel,
    limits = (-5.0,-4.0,-5.0,-4.0))

    # Calculate min and max for setting plot limits
    min_val = min(minimum(etest_pred), minimum(etest_ref))
    max_val = max(maximum(etest_pred), maximum(etest_ref))

    # Add diagonal reference line
    lines!(ax, [min_val, max_val], [min_val, max_val],
    color=:red,
    linestyle=:dash,
    label="Perfect Prediction")

    # Plot scatter with error bars
    errorbars!(ax, etest_ref, etest_pred, qhat_scored,
    whiskerwidth=1,  # Width of error bar caps
    color=:cyan3)

    # Scatter plot of points
    scatter!(ax, etest_ref, etest_pred,
    color=:teal,
    markersize=10)

    # Set equal aspect ratio
    #ax.aspect = DataAspect()

    # Add legend
    axislegend(ax)

    return fig
end

function uncertainty_vs_residuals(uncertainty, residuals;
                                 title  ="Uncertainty vs. residuals",
                                 xlabel = "Distance",
                                 ylabel = "Residuals",
                                 figsize = (600,600))
    fig = Figure(size=figsize)
    ax  = Axis(fig[1,1],
               title=title,
               xlabel=xlabel,
               ylabel=ylabel)

    hlines!(ax, 0.0, color=:red, linestyle=:dash)

    scatter!(ax, uncertainty, residuals, markersize=1)

    #ax.aspect=DataAspect()
    fig
end

In [ ]:
compute_miscalibration_area(alpha_refs, predicted_alphas)